In [153]:
import numpy as np
import urllib
from sklearn import preprocessing
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
from numpy import loadtxt, where
from pylab import scatter, show, legend, xlabel, ylabel
from matplotlib.colors import ListedColormap
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [154]:
#Preprocessed clean version of input file
input_file = "Cleaned_file.csv"

In [155]:
df = pd.read_csv(input_file)

In [156]:
# Printing Data Shape
df.shape

(11670, 6)

In [157]:
df.head(10)

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,5/3/16 21:00,NARCOTICS,True,5,1
1,8,5/3/16 21:35,THEFT,False,1,1
2,9,5/3/16 14:45,ASSAULT,False,3,0
3,8,5/3/16 18:42,BATTERY,False,2,1
4,9,5/3/16 19:30,CRIMINAL DAMAGE,False,3,1
5,11,5/3/16 13:30,OTHER OFFENSE,False,2,0
6,1,5/3/16 19:30,THEFT,False,1,1
7,1,5/3/16 9:11,DECEPTIVE PRACTICE,False,1,0
8,1,5/3/16 6:30,THEFT,False,1,1
9,16,5/3/16 18:30,THEFT,False,1,0


In [158]:
#dataset = np.loadtxt(input_file, delimiter=",")

In [159]:
# Giving numeric values for text data
primaryType = preprocessing.LabelEncoder()
arrest = preprocessing.LabelEncoder()
#date = preprocessing.LabelEncoder()

In [160]:
df["Primary Type"] = primaryType.fit_transform(df["Primary Type"])
df["Arrest"] = arrest.fit_transform(df["Arrest"])
#df["Date"] = date.fit_transform(df["Date"])

In [161]:
df['Date'] = df['Date'].apply(lambda x: x.split('/')[0])

In [162]:
df.head(10)

,District,Date,Primary Type,Arrest,Crime Weightage,Label
0,11,5,16,1,5,1
1,8,5,28,0,1,1
2,9,5,1,0,3,0
3,8,5,2,0,2,1
4,9,5,6,0,3,1
5,11,5,22,0,2,0
6,1,5,28,0,1,1
7,1,5,8,0,1,0
8,1,5,28,0,1,1
9,16,5,28,0,1,0


In [163]:
#df["Date"] = date.inverse_transform(df["Date"])

In [164]:
#df.head()

In [165]:
array = df.values

In [166]:
#columns = ["District", "Date", "Primary Type", "Arrest", "Crime Weightage"]

In [167]:
# Features
X = list(array[:,0:5])

In [168]:
# Target
Y = list(array[:,5])

In [169]:
totalTarget = df["Label"].count()

# Count of 0s in the target
zeroCount = df[(df["Label"] == 0)].count()[0]

# Count of 1s in the target
oneCount = df[(df["Label"] == 1)].count()[0]

#Probability of 0s in the target
probZero = zeroCount / totalTarget

#Probability of 1s in the target
probOne = oneCount / totalTarget

print("Probability of 0s in the Target ", '%0.3f'%(probZero))
print("Probability of 1s in the Target ", '%0.3f'%(probOne))

Probability of 0s in the Target  0.588
Probability of 1s in the Target  0.412


#### Since probability for 0s in the target is higher than 1s, we'll use 0.59 as out baseline accuracy.

#### Splitting Train and Test Data 

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.70, random_state=0)

#### So below we can see that Logistic Regression accuracy is best when argument <i>C >= 1.0</i> i.e. 0.61068

In [171]:
# Calculating accuracy of LOGISTIC REGRESSION using CROSS_VAL_SCORE
C= [5,2,1,0.1,0.01,0.001,0.002,0.0001,0.00001]
for c in C: 
    clf = LogisticRegression(C = c, max_iter = 1000, random_state=0, solver='sag')
    scoresLR = cross_val_score(clf,X_train,y_train,cv=10,scoring='accuracy')
    print("Accuracy with C = ",c,": %0.5f(+/- %0.5f)" %(scoresLR.mean(), scoresLR.std()*2))

Accuracy with C =  5 : 0.61068(+/- 0.03212)
Accuracy with C =  2 : 0.61068(+/- 0.03212)
Accuracy with C =  1 : 0.61068(+/- 0.03212)
Accuracy with C =  0.1 : 0.61043(+/- 0.03311)
Accuracy with C =  0.01 : 0.60982(+/- 0.03165)
Accuracy with C =  0.001 : 0.60982(+/- 0.02942)
Accuracy with C =  0.002 : 0.60859(+/- 0.02738)
Accuracy with C =  0.0001 : 0.60908(+/- 0.02503)
Accuracy with C =  1e-05 : 0.57334(+/- 0.00987)


#### Best Accuracy with Decision Tree Classifier is 0.99951

In [172]:
# Calculating accuracy of DECISION TREE CLASSIFIER using CROSS_VAL_SCORE
dtree = tree.DecisionTreeClassifier()
scoresDT = cross_val_score(dtree,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Decision Tree: %0.5f (+/- %0.5f)" % (scoresDT.mean(), scoresDT.std() * 2))

Accuracy with Decision Tree: 0.99976 (+/- 0.00098)


#### Best Accuracy with Naive Bayes is 0.63920

In [ ]:
# Calculating accuracy of NAIVE BAYES using CROSS_VAL_SCORE
gnb = GaussianNB()
scores = cross_val_score(gnb,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Gaussian Naive Bayes: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

Accuracy with Gaussian Naive Bayes: 0.63920 (+/- 0.03629)


#### Best Accuracy with MLPC Classifier is when <i>learning_rate = 0.003</i> i.e. 0.85161

In [ ]:
L = [0.001,0.002,0.003,0.004,0.005,0.006,0.0001,0.00001, 0.000001]
for l in L: 
    MLPC = MLPClassifier(random_state=0, learning_rate='adaptive',max_iter=10000, learning_rate_init=l)
    scoresMLPC = cross_val_score(MLPC,X_train,y_train,cv=10,scoring='accuracy')
    print("Accuracy with MLPC Classifier with",l,": %0.5f (+/- %0.5f)" % (scoresMLPC.mean(), scoresMLPC.std() * 2))

Accuracy with MLPC Classifier with 0.001 : 0.82578 (+/- 0.03658)
Accuracy with MLPC Classifier with 0.002 : 0.83987 (+/- 0.03972)
Accuracy with MLPC Classifier with 0.003 : 0.85161 (+/- 0.05520)
Accuracy with MLPC Classifier with 0.004 : 0.82946 (+/- 0.05049)
Accuracy with MLPC Classifier with 0.005 : 0.83986 (+/- 0.05623)
Accuracy with MLPC Classifier with 0.006 : 0.84562 (+/- 0.05957)


#### Best Accuracy with Support Vector Machines is 0.96523

In [ ]:
svclf = SVC(cache_size=1000) # Increased Cache size for larger datasets
scores = cross_val_score(svclf,X_train,y_train,cv=10,scoring='accuracy')
print("Accuracy with Support Vector Machines: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))

#### Accuracy of the various models in the order of increasing accuracy:
1. Logistic Regression     : 0.61068
2. Naive Bayes             : 0.63920
3. MLPC Classifer          : 0.85161
4. Support Vector Machines : 0.96523
5. Decision Tree Classifier: 0.99951


#### So comparing the accuracy for all the models, we found Support Vector Machines to be the best for our dataset as it gives the best accuracy of  0.96523. But since SVM and MLPC models runs very slow on Large Datasets, we preferred to use  

In [ ]:
'''Y_pred    = clf.predict(X_test)
Y_score   = clf.decision_function(X_test)
accuracy  = metrics.accuracy_score(y_test, Y_pred)
precision = metrics.precision_score(y_test, Y_pred)
recall    = metrics.recall_score(y_test, Y_pred)
AUC       = metrics.roc_auc_score(y_test, Y_score)'''